In [8]:
import yaml
from pathlib import Path
import os

def check_dataset(dict):
    """ check what dataset is used for training from data_dict and download it. This function is from util/general.py
    """
    # Download dataset if not found locally
    val, s = dict.get('val'), dict.get('download')
    if val and len(val):
        val = [Path(x).resolve() for x in (val if isinstance(val, list) else [val])]  # val path
        if not all(x.exists() for x in val):
            print('\nWARNING: Dataset not found, nonexistent paths: %s' % [str(x) for x in val if not x.exists()])
            if s and len(s):  # download script
                if s.startswith('http') and s.endswith('.zip'):  # URL
                    f = Path(s).name  # filename
                    print(f'Downloading {s} ...')
                    torch.hub.download_url_to_file(s, f)
                    r = os.system(f'unzip -q {f} -d ../ && rm {f}')  # unzip
                elif s.startswith('bash '):  # bash script
                    print(f'Running {s} ...')
                    r = os.system(s)
                else:  # python script
                    r = exec(s)  # return None
                print('Dataset autodownload %s\n' % ('success' if r in (0, None) else 'failure'))  # print result
            else:
                raise Exception('Dataset not found.')

In [9]:
with open("data/coco.yaml") as f:
    data_dict = yaml.safe_load(f)  # data dict

In [10]:
check_dataset(data_dict)


Running bash data/scripts/get_coco.sh ...
Dataset autodownload success



In [ ]:
import sagemaker
from sagemaker import get_execution_role

# SageMaker を活用するための権限が付与された Role を準備します。
role = get_execution_role()

sagemaker_session = sagemaker.Session()
input_coco = sagemaker_session.upload_data(path='../coco', key_prefix='coco')